# TRAINING

In [33]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [34]:
from sagemaker.tensorflow import TensorFlow

hyperparams={'epochs'              : 1,
             'layers'              : 'heads', # heads or all
             'pretrained_weight'   : 'imagenet', # imagenet, coco
            }

bucket_name = sagemaker_session.default_bucket()
output_path = f's3://{bucket_name}/jobs'

In [26]:
import os

os.listdir("train/")

['.ipynb_checkpoints',
 'annotations.json',
 'requirements.txt',
 'Mask_RCNN',
 'model.py',
 'utils.py',
 'model_serving.py',
 'preprocessor.py',
 'train.py']

In [35]:
tf_estimator = TensorFlow(entry_point          = 'train.py', 
                          source_dir           = 'train/',
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          instance_count       = 1, 
                          instance_type        = 'ml.p3.2xlarge',
                          framework_version    = '1.15.3', 
                          py_version           = 'py3',
                          script_mode          = True,
                          sagemaker_session    = sagemaker_session,
                          hyperparameters      = hyperparams)

In [36]:
import time
job_name=f'tensorflow-maskrcnn-gpu-vehicle-parts-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
tf_estimator.fit({'training'  : "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_train",
                  'validation': "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_valid",
                  }, 
                 job_name = job_name)

2021-08-28 12:28:48 Starting - Starting the training job...
2021-08-28 12:28:50 Starting - Launching requested ML instancesProfilerReport-1630153722: InProgress
......
2021-08-28 12:30:18 Starting - Preparing the instances for training......
2021-08-28 12:31:18 Downloading - Downloading input data
2021-08-28 12:31:18 Training - Downloading the training image......
2021-08-28 12:32:19 Training - Training image download completed. Training in progress...WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2021-08-28 12:32:23,158 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-08-28 12:32:24,802 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/usr/bin/python3 -m pip install -r requirements.txt
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-lin

# DEPLOYMENT

In [39]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [40]:
from sagemaker.tensorflow.model import TensorFlowModel

The last thing to do is upload the, now in ProtoBuf form, model to S3, the AWS service for cloud storage.

In [41]:
artifact_dir = "s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-maskrcnn-gpu-vehicle-parts-2021-08-28-12-28-42/output/model.tar.gz"

In [42]:
estimator = TensorFlowModel(model_data= artifact_dir,
                            role=role, 
                            framework_version="1.15.3",
                            entry_point = 'inference.py',
                            source_dir="serve/")

In [43]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

In [64]:
predictor

# PREDICTION

In [49]:
import visualizer as visualize
import skimage

In [54]:
import os
import numpy as np
list_files = os.listdir("sample_files") 

In [51]:
list_files

['car wreck_-ef43703c49fb57a010fa9ef1c6c28dbeffd491793e7333bb4299fe4490d95a7f.jpg',
 'car wreck_-2432b535d4b3f5c96f4090d7aa69d76ba64577f4f85c9bb55817af2dd7c65aec.jpg',
 'car wreck_-b5a3c6729b65b9d02d1398e4f46d17ed2cd1f5f26cc97cf720fc6016ca1208a4.jpg',
 'car wreck_-af76c4700ff3dc62083b2b54eb2d107a5c90526f56bcf072f31b662ca15c33ab.jpg',
 'car wreck_-b0a3ab05fed79d0a3e134c71179c1834768e2d2e2f1d402f0cbdae715354e0f2.jpg',
 'car crash_-f44e3172f0c99d67cac362ea118af93fe6a8868c3d6ed189d8983add49c90c13.jpg',
 'car wreck_-16578778651c5856199faf5a3419a63ac033b2073026a47e2491b9c6f5117cb9.jpg',
 'cardamaged-df735878beb4b81768983862a31d4ffd146d7b9512466f74d50dd112bdd35b79.jpg',
 'car wreck_-c9334facae81b43badac6379cc48f73d8420db1f5eb81c736a01a4fb0d7bae0f.jpg']

In [62]:
def detect(model,files):
    for file in files:
        image_path = f"sample_files/{file}"
        
        img = skimage.io.imread(image_path)
        img_arr = np.array(img)
        results = model.predict(img_arr)
        print(results)
        r = results[0]
        visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                    dataset_val.class_names, r['scores'])
        

In [66]:
# detect(predictor,list_files)

# HOUSE CLEANING

In [65]:
predictor.delete_endpoint()